# utils.py

In [2]:
# Manual Label Encoding
def label_encoder_target(data):
    data = data.map({'Insufficient_Weight':0, 'Normal_Weight':1, 'Overweight_Level_I':2, 'Overweight_Level_II':3, 'Obesity_Type_I':4, 'Obesity_Type_II':5, 'Obesity_Type_III':6})
    return data


def encoder(data, label_col, label_col2, label_col3,label_col4, onehot_cols):
    if label_col in data.columns:
        data[label_col] = data[label_col].map({'Insufficient_Weight':0, 'Normal_Weight':1, 'Overweight_Level_I':2, 'Overweight_Level_II':3, 'Obesity_Type_I':4, 'Obesity_Type_II':5, 'Obesity_Type_III':6})
    for col in onehot_cols:
        if col != 'Gender':
            data[col] = data[col].map({'yes':1, 'no':0})
        else:
            data[col] = data[col].map({'Female':0, 'Male':1})
    
    data[label_col2] = data[label_col2].map({'no':0, 'Sometimes':1, 'Frequently':2, 'Always':3})
    data[label_col3] = data[label_col3].map({'Bike':0, 'Walking':1, 'Public_Transportation':2, 'Motorbike':3, 'Automobile':4})
    data[label_col4] = data[label_col4].map({'no':0, 'Sometimes':1, 'Frequently':2, 'Always':3})
    return data

def decoder(data, label_col, label_col2, label_col3, label_col4, onehot_cols):
    # Original mapping dictionary
    reverse_mapping = {'Insufficient_Weight': 0, 'Normal_Weight': 1, 'Overweight_Level_I': 2, 'Overweight_Level_II': 3, 'Obesity_Type_I': 4, 'Obesity_Type_II': 5, 'Obesity_Type_III': 6}
    # Reverse the mapping
    reverse_mapping = {v: k for k, v in reverse_mapping.items()}
    # Use the reversed mapping to decode
    data[label_col] = data[label_col].map(reverse_mapping)

    # Reverse mapping for onehot_cols
    for col in onehot_cols:
        if col != 'Gender':
            data[col] = data[col].map({1: 'yes', 0: 'no'})
        else:
            data[col] = data[col].map({0: 'Female', 1: 'Male'})

    # Reverse mapping for label_col2
    data[label_col2] = data[label_col2].map({0: 'no', 1: 'Sometimes', 2: 'Frequently', 3: 'Always'})
    # Reverse mapping for label_col3
    data[label_col3] = data[label_col3].map({0: 'Bike', 1: 'Walking', 2: 'Public_Transportation', 3: 'Motorbike', 4: 'Automobile'})
    # Reverse mapping for label_col4
    data[label_col4] = data[label_col4].map({0: 'no', 1: 'Sometimes', 2: 'Frequently', 3: 'Always'})

    return data



# print(A.columns)

# Model


In [5]:
import pandas as pd
import numpy as np
import optuna
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
#current working directory
import os
os.getcwd()
setwd = os.chdir('/Users/arham/Downloads/Kaggle')

onehot = ['Gender' ,'family_history_with_overweight','FAVC','SMOKE','SCC']



# Load the labeled training data (A) and test data (B)
A = pd.read_csv('train.csv')  
B = pd.read_csv('test.csv')   
S = pd.read_csv('ObesityDataSet_raw_and_data_sinthetic.csv')

# Perform encoding if needed
# Assuming encoder and onehot functions are defined or imported properly
A = encoder(A, 'NObeyesdad', 'CAEC', 'MTRANS', 'CALC', onehot)
C = encoder(S, 'NObeyesdad', 'CAEC', 'MTRANS', 'CALC', onehot)
B = encoder(B, 'NObeyesdad', 'CAEC', 'MTRANS', 'CALC', onehot)

# Combine A and C to A 
A = pd.concat([A, C], axis=0)

# Remove duplicates
A = A.drop_duplicates()

# split A into A and A_test
A, A_test = train_test_split(A, test_size=0.2, random_state=42)




# Define scale_cols before scaling
scale_cols = ['Age','Height', 'Weight','FCVC','NCP','CH2O','FAF','TUE']

# Min-max scaling 
for c in scale_cols:  
    A[c] = A[c].pow(0.5)
    B[c] = B[c].pow(0.5) 
    A_test[c] = A_test[c].pow(0.5)

# Split the labeled training data into features (X) and target (y)
X_train = A.drop(columns=['NObeyesdad'])  
y_train = A['NObeyesdad']  

# Define XGBClassifier with parameters # optuna  with 10 iterations
parameters = {'max_depth': 6, 'learning_rate': 0.03617694379652588, 'n_estimators': 629, 'subsample': 0.8204774268264704, 'colsample_bytree': 0.8750737854215174}
model = XGBClassifier(**parameters)

# Fit the model on the entire dataset
model.fit(X_train, y_train)

# Make predictions on the test set
y_test_pred = model.predict(A_test.drop(columns=['NObeyesdad']))


# accuracy

accuracy = accuracy_score(A_test['NObeyesdad'], y_test_pred)
# # Create submission DataFrame
# submission = pd.DataFrame({'id': B['id'], 'NObeyesdad': y_test_pred})

# # Decode NObeyesdad
# reverse_mapping = {
#     0: 'Insufficient_Weight',
#     1: 'Normal_Weight',
#     2: 'Overweight_Level_I',
#     3: 'Overweight_Level_II',
#     4: 'Obesity_Type_I',
#     5: 'Obesity_Type_II',
#     6: 'Obesity_Type_III'
# }
# submission['NObeyesdad'] = submission['NObeyesdad'].map(reverse_mapping)

# # Save submission to CSV
# timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M")
# submission.to_csv(f'submission_{timestamp}.csv', index=False)


accuracy

0.9135478222805865